In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense,Dropout

# 1 - Data Preprocessing

In [8]:
def processar_dados_ano(caminho_arquivo, escolas_comuns):
    df = pd.read_csv(caminho_arquivo, encoding='latin1', delimiter=';')
    df = df[df['CO_ENTIDADE'].isin(escolas_comuns)] 
    colunas_para_remover = [col for col in df.columns if (col.startswith('DS_') or col.startswith('NU_') or col.startswith('DT_') or col.startswith('CO_') or col.startswith('NO_')) and col != 'NU_ANO_CENSO']
    df_limpo = df.drop(columns=colunas_para_remover).drop(columns=['SG_UF'])
    colunas_com_nan = df_limpo.columns[df_limpo.isnull().any()].tolist()
    return df_limpo, colunas_com_nan

diretorio = 'dataset/'
primeiro_ano = 2014
caminho_arquivo = f'{diretorio}microdados_ed_basica_{primeiro_ano}_alagoas.csv'
df_primeiro_ano = pd.read_csv(caminho_arquivo, encoding='latin1', delimiter=';')
escolas_comuns = set(df_primeiro_ano['CO_ENTIDADE'].unique())

for ano in range(primeiro_ano + 1, 2022):
    caminho_arquivo = f'{diretorio}microdados_ed_basica_{ano}_alagoas.csv'
    df = pd.read_csv(caminho_arquivo, encoding='latin1', delimiter=';')
    escolas_comuns = escolas_comuns.intersection(set(df['CO_ENTIDADE'].unique()))

dfs = [] 

for ano in range(primeiro_ano, 2022):
    caminho_arquivo = f'{diretorio}microdados_ed_basica_{ano}_alagoas.csv'
    df_limpo, colunas_com_nan = processar_dados_ano(caminho_arquivo, escolas_comuns)
    dfs.append(df_limpo) 
    print(f"Resumo dos dados de {ano}:")
    print(colunas_com_nan)
    print("--------------------------------------------------------------------\n")
    print(df_limpo.describe())

Resumo dos dados de 2014:
[]
--------------------------------------------------------------------

       NU_ANO_CENSO  TP_DEPENDENCIA  TP_CATEGORIA_ESCOLA_PRIVADA  \
count        2956.0     2956.000000                  2956.000000   
mean         2014.0        3.000677                     0.149188   
std             0.0        0.488791                     0.506208   
min          2014.0        1.000000                     0.000000   
25%          2014.0        3.000000                     0.000000   
50%          2014.0        3.000000                     0.000000   
75%          2014.0        3.000000                     0.000000   
max          2014.0        4.000000                     4.000000   

       TP_LOCALIZACAO  TP_LOCALIZACAO_DIFERENCIADA  TP_SITUACAO_FUNCIONAMENTO  \
count     2956.000000                  2956.000000                2956.000000   
mean         1.514208                     0.088295                   1.044317   
std          0.499883                     0.4

In [9]:
df_consolidado = pd.concat(dfs)
df_consolidado.to_csv('df_consolidado.csv', index=False)

In [15]:
split_percentage = 0.9
split_index = int(split_percentage * len(df_consolidado))
train_data = df_consolidado.iloc[:split_index]
test_data = df_consolidado.iloc[split_index:]

In [17]:
#Normalizando os dados
colunas_continuas = [col for col in df_consolidado.columns if col.startswith('QT_')]

scaler_continuas = MinMaxScaler()
scaler_continuas.fit(train_data[colunas_continuas])
scaled_train_data = scaler_continuas.transform(train_data[colunas_continuas])
scaled_test_data = scaler_continuas.transform(test_data[colunas_continuas])

In [12]:
# correlacao = df_consolidado.corr()

# limiar_correlacao = 0.62
# correlacoes_fortes = correlacao[(np.abs(correlacao) > limiar_correlacao) & (correlacao != 1.0)]

# variaveis_fortes = correlacoes_fortes.dropna(axis=0, how='all').dropna(axis=1, how='all')
# variaveis_fortes_lista = variaveis_fortes.index.tolist()

# variaveis_fortes.to_csv('correlacao.csv', index=True)
# print(len(variaveis_fortes_lista))

# 2 - Creating and training the Model